In [0]:
members_df = spark.sql("SELECT * FROM members")

bookings_df = spark.sql("SELECT * FROM bookings")

facilities_df = spark.sql("SELECT * FROM facilities")

members_df.show()

bookings_df.show()

facilities_df.show()

+-----+---------+---------+--------------------+-------+--------------+-------------+-------------------+
|memid|  surname|firstname|              adress|zipcode|     telephone|recommendedby|           joindate|
+-----+---------+---------+--------------------+-------+--------------+-------------+-------------------+
|    0|    GUEST|    GUEST|               GUEST|      0|(000) 000-0000|         null|2012-07-01 00:00:00|
|    1|    Smith|   Darren|8 Bloomsbury Clos...|   4321|  555-555-5555|         null|2012-07-02 12:02:05|
|    2|    Smith|    Tracy|8 Bloomsbury Clos...|   4321|  555-555-5555|         null|2012-07-02 12:08:23|
|    3|   Rownam|      Tim|23 Highway Way, B...|  23423|(844) 693-0723|         null|2012-07-03 09:32:15|
|    4| Joplette|   Janice|20 Crossing Road,...|    234|(833) 942-4710|            1|2012-07-03 10:25:05|
|    5|  Butters|   Gerald|1065 Huntingdon A...|  56754|(844) 078-4130|            1|2012-07-09 10:44:09|
|    6|    Tracy|   Burton|3 Tunisia Drive, ..

In [0]:
# 1 How can you produce a list of the start times for bookings by members named 'David Farrell'?


from pyspark.sql.functions import col

df = bookings_df.join(members_df, "memid").filter((col("firstname") == "David") & (col("surname") == "Farrell")).select("starttime")

df.show()


+-------------------+
|          starttime|
+-------------------+
|2012-09-18 09:00:00|
|2012-09-18 17:30:00|
|2012-09-18 13:30:00|
|2012-09-18 20:00:00|
|2012-09-19 09:30:00|
|2012-09-19 15:00:00|
|2012-09-19 12:00:00|
|2012-09-20 15:30:00|
|2012-09-20 11:30:00|
|2012-09-20 14:00:00|
|2012-09-21 10:30:00|
|2012-09-21 14:00:00|
|2012-09-22 08:30:00|
|2012-09-22 17:00:00|
|2012-09-23 08:30:00|
|2012-09-23 17:30:00|
|2012-09-23 19:00:00|
|2012-09-24 08:00:00|
|2012-09-24 16:30:00|
|2012-09-24 12:30:00|
+-------------------+
only showing top 20 rows



In [0]:
# 2 How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.

from pyspark.sql.functions import col

df = (
    bookings_df.join(facilities_df, "facid")
    .filter((col("name").isin("Tennis Court 1", "Tennis Court 2")) & 
            (col("starttime") >= "2012-09-21") & 
            (col("starttime") < "2012-09-22")) 
    .select("starttime", "name")
    .orderBy("starttime")

)


df.show(truncate=False)


+-------------------+--------------+
|starttime          |name          |
+-------------------+--------------+
|2012-09-21 08:00:00|Tennis Court 1|
|2012-09-21 08:00:00|Tennis Court 2|
|2012-09-21 09:30:00|Tennis Court 1|
|2012-09-21 10:00:00|Tennis Court 2|
|2012-09-21 11:30:00|Tennis Court 2|
|2012-09-21 12:00:00|Tennis Court 1|
|2012-09-21 13:30:00|Tennis Court 1|
|2012-09-21 14:00:00|Tennis Court 2|
|2012-09-21 15:30:00|Tennis Court 1|
|2012-09-21 16:00:00|Tennis Court 2|
|2012-09-21 17:00:00|Tennis Court 1|
|2012-09-21 18:00:00|Tennis Court 2|
+-------------------+--------------+



In [0]:
# 3 How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).

from pyspark.sql import functions

df = (
    members_df.alias("mems") 
    .join(members_df.alias("recs"), col("mems.recommendedby") == col("recs.memid"), "left") 
    .select(
        col("mems.firstname").alias("memfname"), 
        col("mems.surname").alias("memsname"), 
        col("recs.firstname").alias("recfname"), 
        col("recs.surname").alias("recsname")
    ) 
    .orderBy(col("memsname"), col("memfname"))

)


df.show()


+---------+---------+---------+--------+
| memfname| memsname| recfname|recsname|
+---------+---------+---------+--------+
| Florence|    Bader|   Ponder|Stibbons|
|     Anne|    Baker|   Ponder|Stibbons|
|  Timothy|    Baker|   Jemima| Farrell|
|      Tim|   Boothe|      Tim|  Rownam|
|   Gerald|  Butters|   Darren|   Smith|
|     Joan|   Coplin|  Timothy|   Baker|
|    Erica|  Crumpet|    Tracy|   Smith|
|    Nancy|     Dare|   Janice|Joplette|
|    David|  Farrell|     null|    null|
|   Jemima|  Farrell|     null|    null|
|    GUEST|    GUEST|     null|    null|
|  Matthew|  Genting|   Gerald| Butters|
|     John|     Hunt|Millicent| Purview|
|    David|    Jones|   Janice|Joplette|
|  Douglas|    Jones|    David|   Jones|
|   Janice| Joplette|   Darren|   Smith|
|     Anna|Mackenzie|   Darren|   Smith|
|  Charles|     Owen|   Darren|   Smith|
|    David|   Pinker|   Jemima| Farrell|
|Millicent|  Purview|    Tracy|   Smith|
+---------+---------+---------+--------+
only showing top

In [0]:
from pyspark.sql.functions import col, concat_ws

# 4 How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.

df = (
    bookings_df
    .join(members_df, "memid")  
    .join(facilities_df, "facid") 
    .filter(col("name").isin("Tennis Court 1", "Tennis Court 2"))  
    .select(
        concat_ws(" ", col("firstname"), col("surname")).alias("member"),
        col("name").alias("facility")
    )
    .distinct() 
    .orderBy("member", "facility")  
)

df.show()


+--------------+--------------+
|        member|      facility|
+--------------+--------------+
|    Anne Baker|Tennis Court 1|
|    Anne Baker|Tennis Court 2|
|  Burton Tracy|Tennis Court 1|
|  Burton Tracy|Tennis Court 2|
|  Charles Owen|Tennis Court 1|
|  Charles Owen|Tennis Court 2|
|  Darren Smith|Tennis Court 2|
| David Farrell|Tennis Court 1|
| David Farrell|Tennis Court 2|
|   David Jones|Tennis Court 1|
|   David Jones|Tennis Court 2|
|  David Pinker|Tennis Court 1|
| Douglas Jones|Tennis Court 1|
| Erica Crumpet|Tennis Court 1|
|Florence Bader|Tennis Court 1|
|Florence Bader|Tennis Court 2|
|   GUEST GUEST|Tennis Court 1|
|   GUEST GUEST|Tennis Court 2|
|Gerald Butters|Tennis Court 1|
|Gerald Butters|Tennis Court 2|
+--------------+--------------+
only showing top 20 rows



In [0]:
# 5 How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.

from pyspark.sql.functions import col, concat_ws

df = (
    members_df.alias("mems")
    .join(
        members_df.alias("recs"),
        col("mems.recommendedby") == col("recs.memid"),
        "left" 
    )
    .select(
        concat_ws(" ", col("mems.firstname"), col("mems.surname")).alias("member"),
        concat_ws(" ", col("recs.firstname"), col("recs.surname")).alias("recommender")
    )
    .distinct() 
    .orderBy("member") 
)

df.show()


+--------------------+---------------+
|              member|    recommender|
+--------------------+---------------+
|      Anna Mackenzie|   Darren Smith|
|          Anne Baker|Ponder Stibbons|
|        Burton Tracy|               |
|        Charles Owen|   Darren Smith|
|        Darren Smith|               |
|       David Farrell|               |
|         David Jones|Janice Joplette|
|        David Pinker| Jemima Farrell|
|       Douglas Jones|    David Jones|
|       Erica Crumpet|    Tracy Smith|
|      Florence Bader|Ponder Stibbons|
|         GUEST GUEST|               |
|      Gerald Butters|   Darren Smith|
|    Henrietta Rumney|Matthew Genting|
|Henry Worthington...|    Tracy Smith|
| Hyacinth Tupperware|               |
|          Jack Smith|   Darren Smith|
|     Janice Joplette|   Darren Smith|
|      Jemima Farrell|               |
|         Joan Coplin|  Timothy Baker|
+--------------------+---------------+
only showing top 20 rows



In [0]:
# 6 Produce a count of the number of recommendations each member has made. Order by member ID.

from pyspark.sql import functions

df = (
    members_df
    .filter(col("recommendedby").isNotNull()) 
    .groupBy("recommendedby") 
    .agg(functions.count("*").alias("count"))
    .orderBy("recommendedby") 
)

df.show()

+-------------+-----+
|recommendedby|count|
+-------------+-----+
|            1|    5|
|            2|    3|
|            3|    1|
|            4|    2|
|            5|    1|
|            6|    1|
|            9|    2|
|           11|    1|
|           13|    2|
|           15|    1|
|           16|    1|
|           20|    1|
|           30|    1|
+-------------+-----+



In [0]:
# 7 Produce a list of the total number of slots booked per facility. For now, just produce an output table consisting of facility id and slots, sorted by facility id.
from pyspark.sql import functions

df = (
    bookings_df
    .select("facid", "slots")
    .groupBy("facid")
    .agg(sum("slots").alias("Total Slot"))
    .orderBy("facid")
    )

df.show()

+-----+----------+
|facid|Total Slot|
+-----+----------+
|    0|      1320|
|    1|      1278|
|    2|      1209|
|    3|       830|
|    4|      1404|
|    5|       228|
|    6|      1104|
|    7|       908|
|    8|       911|
+-----+----------+



In [0]:
# 8 Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.

from pyspark.sql import functions as f

df= (
    bookings_df
    .select("facid", "slots")
    .filter((f.col("starttime") >= '2012-09-01') & (f.col("starttime") < '2012-10-01'))
    .groupBy("facid")
    .agg(sum("slots").alias("Total Slots"))
    .orderBy("Total Slots")
)


df.show()

+-----+-----------+
|facid|Total Slots|
+-----+-----------+
|    5|        122|
|    3|        422|
|    7|        426|
|    8|        471|
|    6|        540|
|    2|        570|
|    1|        588|
|    0|        591|
|    4|        648|
+-----+-----------+



In [0]:
# 9 Produce a list of the total number of slots booked per facility per month in the year of 2012. Produce an output table consisting of facility id and slots, sorted by the id and month.

from pyspark.sql import functions as f

df = (
    bookings_df
    .filter(f.year("starttime") == 2012)  
    .select(
        "facid",
        f.month("starttime").alias("month"),  
        "slots"
    )
    .groupBy("facid", "month")  
    .agg(f.sum("slots").alias("Total Slots"))  
    .orderBy("facid", "month")  
)

df.show()


+-----+-----+-----------+
|facid|month|Total Slots|
+-----+-----+-----------+
|    0|    7|        270|
|    0|    8|        459|
|    0|    9|        591|
|    1|    7|        207|
|    1|    8|        483|
|    1|    9|        588|
|    2|    7|        180|
|    2|    8|        459|
|    2|    9|        570|
|    3|    7|        104|
|    3|    8|        304|
|    3|    9|        422|
|    4|    7|        264|
|    4|    8|        492|
|    4|    9|        648|
|    5|    7|         24|
|    5|    8|         82|
|    5|    9|        122|
|    6|    7|        164|
|    6|    8|        400|
+-----+-----+-----------+
only showing top 20 rows



In [0]:
# 10 Find the total number of members (including guests) who have made at least one booking.

from pyspark.sql import functions as f

df = (
    bookings_df
    .select("memid")
    .dropDuplicates()
    .agg(f.count("memid"))
)

df.show()

+------------+
|count(memid)|
+------------+
|          30|
+------------+



In [0]:
# 11 Produce a list of each member name, id, and their first booking after September 1st 2012. Order by member ID.

from pyspark.sql import functions as f

df = (
     bookings_df
    .join(members_df, "memid")  
    .filter(f.col("starttime") >= '2012-09-01') 
    .groupBy("surname", "firstname", "memid")  
    .agg(f.min("starttime").alias("starttime")) 

    .orderBy("memid")  
)

df.show()

+---------+---------+-----+-------------------+
|  surname|firstname|memid|          starttime|
+---------+---------+-----+-------------------+
|    GUEST|    GUEST|    0|2012-09-01 08:00:00|
|    Smith|   Darren|    1|2012-09-01 09:00:00|
|    Smith|    Tracy|    2|2012-09-01 11:30:00|
|   Rownam|      Tim|    3|2012-09-01 16:00:00|
| Joplette|   Janice|    4|2012-09-01 15:00:00|
|  Butters|   Gerald|    5|2012-09-02 12:30:00|
|    Tracy|   Burton|    6|2012-09-01 15:00:00|
|     Dare|    Nancy|    7|2012-09-01 12:30:00|
|   Boothe|      Tim|    8|2012-09-01 08:30:00|
| Stibbons|   Ponder|    9|2012-09-01 11:00:00|
|     Owen|  Charles|   10|2012-09-01 11:00:00|
|    Jones|    David|   11|2012-09-01 09:30:00|
|    Baker|     Anne|   12|2012-09-01 14:30:00|
|  Farrell|   Jemima|   13|2012-09-01 09:30:00|
|    Smith|     Jack|   14|2012-09-01 11:00:00|
|    Bader| Florence|   15|2012-09-01 10:30:00|
|    Baker|  Timothy|   16|2012-09-01 15:00:00|
|   Pinker|    David|   17|2012-09-01 08

In [0]:
# 12 Output the names of all members, formatted as 'Surname, Firstname'

from pyspark.sql import functions as f

df = (
    members_df
    .select(f.concat_ws(", ", f.col("surname"), f.col("firstname")).alias("name"))
)

df.show()


+----------------+
|            name|
+----------------+
|    GUEST, GUEST|
|   Smith, Darren|
|    Smith, Tracy|
|     Rownam, Tim|
|Joplette, Janice|
| Butters, Gerald|
|   Tracy, Burton|
|     Dare, Nancy|
|     Boothe, Tim|
|Stibbons, Ponder|
|   Owen, Charles|
|    Jones, David|
|     Baker, Anne|
| Farrell, Jemima|
|     Smith, Jack|
| Bader, Florence|
|  Baker, Timothy|
|   Pinker, David|
|Genting, Matthew|
| Mackenzie, Anna|
+----------------+
only showing top 20 rows



In [0]:
# 13 Perform a case-insensitive search to find all facilities whose name begins with 'tennis'. Retrieve all columns.

from pyspark.sql import functions as f

df = (
    facilities_df
    .select("*")
    .filter(f.upper(col("name")).ilike('TENNIS%')) 
)

df.show()

+-----+--------------+----------+---------+-------------+------------------+
|facid|          name|membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+--------------+----------+---------+-------------+------------------+
|    0|Tennis Court 1|       5.0|     25.0|      10000.0|             200.0|
|    1|Tennis Court 2|       5.0|     25.0|       8000.0|             200.0|
+-----+--------------+----------+---------+-------------+------------------+



In [0]:
# 14 You've noticed that the club's member table has telephone numbers with very inconsistent formatting. You'd like to find all the telephone numbers that contain parentheses, returning the member ID and telephone number sorted by member ID.

from pyspark.sql import functions as f

df = (
    members_df
    .select("memid", "telephone")
    .filter(f.col("telephone").rlike("[()]"))
)

df.show()

+-----+--------------+
|memid|     telephone|
+-----+--------------+
|    0|(000) 000-0000|
|    3|(844) 693-0723|
|    4|(833) 942-4710|
|    5|(844) 078-4130|
|    6|(822) 354-9973|
|    7|(833) 776-4001|
|    8|(811) 433-2547|
|    9|(833) 160-3900|
|   10|(855) 542-5251|
|   11|(844) 536-8036|
|   13|(855) 016-0163|
|   14|(822) 163-3254|
|   15|(833) 499-3527|
|   20|(811) 972-1377|
|   21|(822) 661-2898|
|   22|(822) 499-2232|
|   24|(822) 413-1470|
|   27|(822) 989-8876|
|   28|(855) 755-9876|
|   29|(855) 894-3758|
+-----+--------------+
only showing top 20 rows



In [0]:
# 15 You'd like to produce a count of how many members you have whose surname starts with each letter of the alphabet. Sort by the letter, and don't worry about printing out a letter if the count is 0.

from pyspark.sql import functions as f

df = (
    members_df
    .select(f.col("surname").substr(1,1).alias("letter"))
    .groupBy("letter")
    .agg(f.count("*").alias("count"))
    .orderBy("letter")
)

df.show()

+------+-----+
|letter|count|
+------+-----+
|     B|    5|
|     C|    2|
|     D|    1|
|     F|    2|
|     G|    2|
|     H|    1|
|     J|    3|
|     M|    1|
|     O|    1|
|     P|    2|
|     R|    2|
|     S|    6|
|     T|    2|
|     W|    1|
+------+-----+



In [0]:
# 16 Produce a list of all the dates in October 2012. They can be output as a timestamp (with time set to midnight) or a date.

from pyspark.sql import functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType
from datetime import datetime

# Initialize Spark Session
spark = SparkSession.builder.getOrCreate()

# Define start and end dates
start_date = datetime(2012, 10, 1)
end_date = datetime(2012, 10, 31)

df = (
    spark.range(1)  # Dummy row
    .select(
        f.explode(
            f.sequence(
                f.lit(start_date).cast(TimestampType()), 
                f.lit(end_date).cast(TimestampType()), 
                f.expr("INTERVAL 1 DAY")
            )
        ).alias("ts")
    )
)

df.show()


+-------------------+
|                 ts|
+-------------------+
|2012-10-01 00:00:00|
|2012-10-02 00:00:00|
|2012-10-03 00:00:00|
|2012-10-04 00:00:00|
|2012-10-05 00:00:00|
|2012-10-06 00:00:00|
|2012-10-07 00:00:00|
|2012-10-08 00:00:00|
|2012-10-09 00:00:00|
|2012-10-10 00:00:00|
|2012-10-11 00:00:00|
|2012-10-12 00:00:00|
|2012-10-13 00:00:00|
|2012-10-14 00:00:00|
|2012-10-15 00:00:00|
|2012-10-16 00:00:00|
|2012-10-17 00:00:00|
|2012-10-18 00:00:00|
|2012-10-19 00:00:00|
|2012-10-20 00:00:00|
+-------------------+
only showing top 20 rows



In [0]:
# 17 Return a count of bookings for each month, sorted by month


from pyspark.sql import functions as f

df = (
    bookings_df
    .select("starttime")
    .groupBy(f.date_trunc("month", f.col("starttime")).alias("month"))
    .agg(f.count("*").alias("count")) 
    .orderBy("month") 
)

df.show()


+-------------------+-----+
|              month|count|
+-------------------+-----+
|2012-07-01 00:00:00|  658|
|2012-08-01 00:00:00| 1472|
|2012-09-01 00:00:00| 1913|
|2013-01-01 00:00:00|    1|
+-------------------+-----+

